Welcome to Baseball Analytics using Python! Some of the core technical concepts we will be covering in this workshop will be:
- Data Visualization Techniques / General Graphing
- Linear Regression
- Pandas DataFrames / Data Manipulation / Data Analysis

Some of the core baseball concepts we will be covering in this workshop will be:
- Sabermetrics
- Run Expectancy / Win Expectancy
- Understanding Career Trajectories

To start, here a list of the important baseball statistics you will need to know for our analysis today:
- Batting Average
- Slugging Percentage (SLG)
- On Base Percentage (OBP)
- On Base + Slugging (OPS)

-------------------------------------------------------------------------

# Introduction to Sabermetrics

Definition:
Sabermetrics is the empirical analysis of baseball, especially baseball statistics that measure in-game activity. 

- Why is this important?

- Examples:
     - wRC+
     - defense-independent ERA
     - FIP
     - OPS+

-------------------------------------------------------------------------

# Why use Plotly?

- Sophistication
- Quality
- Community

-------------------------------------------------------------------------

# Lahman's Database #

Lahman's Database contains batting, fielding, and pitching statistics from 1871 - present while also having standings, team stats, managerial records, postseason statistics, and more. It is an incredibly valuable resource for baseball analytics and that is where our data will be coming from today.

-------------------------------------------------------------------------

We first need to install the libraries and packages we are going to use for our analysis.

In [ ]:
%pip install plotly
%pip install nbformat
%pip install statsmodels
%pip install matplotlib
%pip install numpy
%pip install scipy

-------------------------------------------------------------------------

# Graphing

We will begin by making some simple graphs depicting some general relationships common to baseball. First, install the required graphing modules from the previously installed pacakges:

In [ ]:
import pandas as pd
import plotly.express as px
pd.options.mode.chained_assignment = None

The most important module we installed is Pandas. This allows us to work within DataFrames, which makes our analysis much easier. We'll start by reading in the 'hofbatting' csv file which contains career statistics for each member of the baseball hall of fame.

In [ ]:
hofbatting = pd.read_csv('hofbatting.csv')

Now when we look at our variable 'hofbatting', we will see it is a pandas dataframe:

In [ ]:
hofbatting

To get accustomed to using dataframes, let's create a column that shows us the year in which a player was at the middle of his career called 'MidCareer' and another column which shows us what era of baseball each player played in. We can use the cut function to do this using our new MidCareer variable and then sort the values into their respective eras, or bins. 

In [ ]:
# Create MidCareer and Era Columns
hofbatting['MidCareer'] = (hofbatting['From'] + hofbatting['To']) / 2
hofbatting['Era'] = pd.cut(hofbatting['MidCareer'], bins=(1800, 1900, 1919, 1941, 1960, 1976, 1993, 2000), labels=['19th Century', 'Lively Ball', 'Dead Ball', 'Integration', 'Expansion', 'Free Agency', 'Long Ball'])

Now let's make our first graph. Let's look at how many hall of famers played in each different era of baseball. First we'll create a new dataframe with the frequencies for each era using the value.counts() function. We then use to_frame() and reset_index() to turn our 'freq' variable into a dataframe and reset its index:

In [ ]:
freq = hofbatting['Era'].value_counts().to_frame().reset_index(level=0)
freq = freq.rename(columns={'index': 'Era', 'Era': 'Frequency'})

In [ ]:
freq

Now let's make our graph using plotly:

In [ ]:
bar = px.bar(freq, x=freq['Era'], y=freq['Frequency'])
bar.show()

Now as a pie chart:

In [ ]:
pie = px.pie(freq, values=freq['Frequency'], names=freq['Era'])
pie.show()

Or a dot plot:

In [ ]:
dot = px.scatter(freq, x=freq['Frequency'], y=freq['Era'])
dot.show()

Maybe even a histogram of the distribution. Let's add some trace lines as well to distinguish our bars using plotly's update_traces function:

In [ ]:
hist = px.histogram(hofbatting, x=hofbatting['MidCareer'], nbins=12)
hist.update_traces(marker_line_color = 'Black', marker_line_width = 0.5)
hist.show()

Now let's take a look at a general scatter plot of every hall of fame player and their MidCareer value using the index function:

In [ ]:
scatter = px.scatter(hofbatting, x=hofbatting['MidCareer'].index, y=hofbatting['MidCareer'])
scatter.show()

And now a strip chart of the same data:

In [ ]:
strip = px.strip(hofbatting, x=hofbatting['MidCareer'])
strip.show()

These are some of the more simplistic graphing techniques we can employ using Plotly. Now let's look at some two-factor plotting. For this, we'll look at a player's OPS at the midpoint of their career (using our MidCareer variable):

In [ ]:
twovar = px.scatter(hofbatting, x = hofbatting['MidCareer'], y = hofbatting['OPS'], trendline='lowess')
twovar.show()

Here I've added a LOWESS (locally weighted scatterplot smoothing) trendline to showcase some of Plotly's non-linear trendline capabilites and to generally get a better feel of our relationship.

Let's get even more advanced and create a layered strip chart showing Home Run Rates for each era of baseball. We'll create a new column in our hofbatting dataframe:

In [ ]:
hofbatting['HR Rate'] = hofbatting['HR'] / hofbatting['AB']

And then plot this factor in relationship to each era of baseball:

In [ ]:
hrrateplot = px.strip(hofbatting, x=hofbatting['HR Rate'], y= hofbatting['Era'])
hrrateplot.show()

Let's do the same thing but with a boxplot for a more concrete understanding of our relationship:

In [ ]:
hrratebox = px.box(hofbatting, x=hofbatting['HR Rate'], y=hofbatting['Era'])
hrratebox.show()

-------------------------------------------------------------------------

# Runs vs. Wins

Now that we know how to create graphs easily in plotly, lets dig into linear regression whilst exploring some other parts of our dataset:

In [ ]:
teams = pd.read_csv('Teams.csv')

Let's look at a subset of the teams data and filter it for 21st century teams only:

In [ ]:
myteams = teams[['yearID', "teamID", 'lgID', 'G', 'W', 'L', 'R', 'RA']] [teams['yearID'] > 2000]

Now we make two new variables, one for **Run Differential** and one for **Winning Percentage**:

In [ ]:
myteams['RunDiff'] = myteams['R'] - myteams['RA']
myteams['Winpct'] = myteams['W'] / (myteams['W'] + myteams['L'])

In [ ]:
myteams

Using what we just learned, let's graph the two together and see what kind of relationship these two factors have. Let's also add an **Ordinary Least Squares** trendline:

In [ ]:
plot = px.scatter(myteams, x='RunDiff', y='Winpct', trendline='ols')
plot.show()

This seems to make sense: teams with a good run differential tend to have a higher winning percentage.

Now let's train a model to predict win percentage and number of wins using run differential and a linear regression model. We'll do this by using the Statsmodels package in Python:

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

Now we can make the model:

In [ ]:
model = smf.ols(formula = 'Winpct ~ RunDiff', data = myteams).fit()

We can check model accuracy and fit using the `summary()` function:

In [ ]:
model.summary()

And now we can predict winning percentage and wins using the model we fit:

In [ ]:
myteams['predictedWinpct'] = model.predict()
myteams['predictedW'] = round(myteams['W']+myteams['L'])*myteams['predictedWinpct']

Let's check how our model did:

In [ ]:
myteams[['Winpct', 'predictedWinpct', 'W', 'predictedW']]

Pretty solid. Now we can check the normality of our model by looking at the residuals. These functions also come from statsmodels:

In [ ]:
influence = model.get_influence()
residuals = influence.resid_studentized_internal

Now lets make a graph of our resiudals and highlight any 'outliers', or values higher than 2.8 (an arbitrary value I decided on after looking at the data). To do this, we'll use Plotly, but not in the way that we've learned. We're going to use Plotly's `graph_objects` module, which gives us even more control over our graphing capabilities:

In [ ]:
import numpy as np
import plotly.graph_objs as go

residplot = go.Figure(go.Scatter(
    x = myteams['RunDiff'],
    y = residuals,
    mode = 'markers',
    marker = dict(
       color = 
        pd.Series(np.where(residuals>2.8, 'outlier', np.where(residuals<-2.8, 'outlier', 'normal')))
        .astype(str).map({'normal': 0, 'outlier': 1}),
        colorscale = [[0, 'blue'], [1, 'red']]
       )
     
    )
)
residplot.show()

We use a `series` (pd.Series in the code) to create an array which lists all values greater than or equal to 2.8 as outliers and anything less than 2.8 as normal. We then create a `map()` for the color of our points which maps normal to blue and outlier to red. We now have three teams identified. These teams are ones that the model predicted to have likely underperformed or overperformed based on their run differential and win percentage. 

-------------------------------------------------------------------------

# Career Trajectories #

The last thing we are going to look at is attempting to plot and understand career trajectories in baseball. Many players peak at different ages as baseball is a sport with high variance in season-to-season performance. Let's start by reading in two more csv files from Lahman's database:

In [ ]:
batting = pd.read_csv('Batting.csv')
master = pd.read_csv('Master.csv', encoding='latin-1')

For the purposes of this exercise, we can replace NA values with 0:

In [ ]:
batting['SF'] = batting['SF'].fillna(0)
batting['HBP'] = batting['HBP'].fillna(0)

Now we need to define three functions to make it easy for us to look a player's unique database ID, their birthyear, and then their career statistics per year. Let's start with getting a player's ID:

In [ ]:
def getid(firstname, lastname):
    df = master.loc[(master['nameFirst'] == firstname) & (master['nameLast'] == lastname)]
    id = df['playerID']
    return id

Panda's `loc()` function helps a lot when searching through a DataFrame. Next, we need to get a player's birth year:

In [ ]:
def getbirthyear(playerid):
    df = master.loc[master['playerID'] == playerid]
    if df['birthMonth'].any() >= 7:
        df['birthMonth']
    else:
        df['birthYear'] += 1
    return df['birthYear']

So we create a DataFrame (`df` in the above function) using the `playerid` we find using our `getid()` function and then using this DataFrame, extract the birth year. According the Major League Baseball, a player's age is defined as how old he is as of July 1st, so we change it to account for this caveat.

Our third function will get us a player's yearly statistics, mainly SLG, OBP, and OPS:

In [ ]:
def getstats(playerid):
    df = batting.loc[batting['playerID'] == playerid]
    birthyear = getbirthyear(playerid)
    df['Age'] = df['yearID'] - int(birthyear)
    df['SLG'] = (df['H'] - df['2B'] - df['3B'] - df['HR'] + 2 * df['2B'] + 3 * df['3B'] + 4 * df['HR']) / df['AB']
    df['OBP'] = (df['H'] + df['BB']) / (df['H'] + df['AB'] + df['BB'] + df['SF'])
    df['OPS'] = df['SLG'] + df['OBP']
    stats = df[['Age', 'SLG', 'OBP', 'OPS']]
    return stats

We use the same process we did before by creating a DataFrame within the function, fill it with Age and SLG, OBP, and OPS statistics, and then return the DataFrame. You can see that to get Age, we use the getbirthyear that we created and then we subtract that year from each year of their career (1970 - 1940 = a player who is 30 years old). This is what we get when we combine all of our functions to find Mickey Mantle's career stats:

In [ ]:
getid('Mickey', 'Mantle')


In [ ]:
getstats('mantlmi01')

Voila! We now have careeer statistics for every year Mickey Mantle played. What can we do with this? Well, using the plotting and modeling techniques we've used, we can find a player's peak age of performance. Start by graphing Mantle's OPS vs. Age to give us a general understanding of how he looked over his career:

In [ ]:
mantleplot = px.scatter(getstats('mantlmi01'), x = 'Age', y = 'OPS')
mantleplot.show()

Interesting stuff. Now let's make a model that can predict any player's peak age based off of their OPS per year:

In [ ]:
def careerprojchart(playerid):
    stats = getstats(playerid)
    model = np.poly1d(np.polyfit(stats['Age'], stats['OPS'], 2))
    x = np.linspace(1, 50, 50)
    y = model(x)
    plot = go.Scatter(
        x = stats['Age'],
        y = stats['OPS'],
        mode = 'markers',
        marker = go.scatter.Marker(color='rgb(255, 127, 14)'),
        name = 'Data',
    )
    line = go.Scatter(
        x = x,
        y = y,
        mode = 'lines',
        marker = go.scatter.Marker(color = 'rgb(31, 119, 180)'),
        name = 'Fit'
    )
    data = [plot, line]
    fig = go.Figure(data = data)
    max_y = int(max(y))
    max_x = (x[y.argmax(max_y)])
    fig.add_vline(x = max_x, line_dash = 'dash', line_color = 'grey')
    fig.add_hline(y = max(y), line_dash = 'dash', line_color = 'grey')
    fig.update_yaxes(range = [(min(stats['OPS'] - 0.1)), (max(stats['OPS'] + 0.1))])
    fig.update_xaxes(range = [(min(stats['Age'] - 1)), (max(stats['Age']+ 1))])
    fig.update_layout(
        xaxis = dict(tickmode = 'linear', dtick = 2), 
        xaxis_title = 'Age', yaxis_title = 'OPS', 
        title = 'Career Projection for {}'.format(playerid))
    return fig

Let's walk through this function: we start by calling the `getstats()` function so we have data to work with. We then use Numpy's `polyfit()` function to create a least squares polynomial fit of degree 2 for the relationship between Age and OPS. We use degree 2 because after looking at Mantle's career plot and having a general understanding of how athlete's careers stereotypically pan out, we know that a quadratic function will be best for our fit. 

We then use Numpy's `poly1d()` function which is defined as a 'convenience class' and lets us run natural operations which may take on their customary form in code. We apply our model using numpy's `linspace()` function which gives us a series of points for graph so we are able to plot our model in the line section of our plotting function. We first make a scatter plot for our datapoints and then our fitted quadratic line and then graph these two together using the `go.Figure()` function in plotly. 

Lastly, we create dashed lines for the peak values of Age and OPS,update the range of our graphs to fit each individual career (format the graph to look normal for any input player), and add labels and a title.

In [ ]:
careerprojchart('mantlmi01')

Trying out the function for some other players:

In [ ]:
getid('David', 'Ortiz')

In [ ]:
careerprojchart('ortizda01')

In [ ]:
getid('Sammy', 'Sosa')

In [ ]:
careerprojchart('sosasa01')

-------------------------------------------------------------------------